In [1]:
import torch
import warnings
warnings.filterwarnings("ignore")
import dgl
import dgllife
import rdkit
from dgllife.utils import smiles_to_bigraph
from featurize import featurize_atoms,featurize_bonds

In [2]:
from function import NN_ECS,cal_pressure,cal_energy

In [3]:
print(f"torch version: {torch.__version__}")
print(f"rdkit version: {rdkit.__version__}")
print(f"dgl version: {dgl.__version__}")
print(f"dgllife version: {dgllife.__version__}")

torch version: 2.3.0+cu121
rdkit version: 2023.09.4
dgl version: 2.2.1+cu121
dgllife version: 0.3.2


In [4]:
#load the NN-ECS model
model=torch.load('model.pt')
model.to('cpu');

In [5]:
#transform the SMILES string to the molecular graph in dgl
g_tar=smiles_to_bigraph(smiles='C=C(C(F)(F)F)F',node_featurizer=featurize_atoms,edge_featurizer=featurize_bonds)  #R1234yf  the test target fluid
g_ref=smiles_to_bigraph(smiles='C(=C/F)\C(F)(F)F',node_featurizer=featurize_atoms,edge_featurizer=featurize_bonds)  #R1234ze(E) the reference fluid

In [6]:
t=torch.tensor([300,310,400,]);
d=torch.tensor([11,10,5]);
tc=367.85;dc=4.18;

In [7]:
#calculate the pressure
p=cal_pressure(model,g_tar,g_ref,t,d,tc,dc)  #MPa

In [8]:
p

tensor([45.6661, 15.7411,  6.3130], grad_fn=<MulBackward0>)

In [9]:
#calculate the residual entropy and residual enthalpy
sr,hr=cal_energy(model,g_tar,g_ref,t,d,tc,dc)   #sr J/mol ht J/mol/k

In [10]:
sr     #sr kJ/mol  

tensor([-0.0375, -0.0315, -0.0127])

In [11]:
hr  #hr kJ/mol/k

tensor([-16.4142, -17.2603, -10.2271])